In [1]:
import psycopg2
import pandas as pd

In [ ]:
# Load data into a dictionary of DataFrames
dfs = {table: pd.read_sql(f"SELECT * FROM {table};", conn) for table in tables}
print(len(dfs))
print(dfs["dim_customers"])

conn.close()

/var/folders/7_/vzxv0s1j43n3s8qd35q3b6_m0000gn/T/ipykernel_1734/3501130695.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs = {table: pd.read_sql(f"SELECT * FROM {table};", conn) for table in tables}


5
    customer_id       customer_name            city currency
0        789101    Costco Wholesale  New Jersey, US      USD
1        789102        Vijay Stores       Ahmedabad      INR
2        789103        Vijay Stores        Vadodara      INR
3        789121            Coolblue       Ahmedabad      INR
4        789122            Coolblue        Vadodara      INR
5        789201           Rel Fresh  New Jersey, US      USD
6        789202           Rel Fresh       Ahmedabad      INR
7        789203           Rel Fresh        Vadodara      INR
8        789220            ShopRite  New Jersey, US      USD
9        789221        Atlas Stores       Ahmedabad      INR
10       789301            Foodtown  New Jersey, US      USD
11       789303   Expression Stores        Vadodara      INR
12       789320  Whole Foods Market  New Jersey, US      USD
13       789321      Chiptec Stores       Ahmedabad      INR
14       789401             Wegmans  New Jersey, US      USD
15       789402       

In [10]:
dfs["dim_customers"].head()
#dfs["fact_order_line"].info()

,customer_id,customer_name,city,currency
0,789101,Costco Wholesale,"New Jersey, US",USD
1,789102,Vijay Stores,Ahmedabad,INR
2,789103,Vijay Stores,Vadodara,INR
3,789121,Coolblue,Ahmedabad,INR
4,789122,Coolblue,Vadodara,INR


In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [22]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent


In [33]:
merged_df = dfs["fact_order_line"] \
    .merge(dfs["dim_products"], on="product_id", how="left") \
    .merge(dfs["dim_customers"], on="customer_id", how="left")

In [38]:
agent = create_pandas_dataframe_agent(
    llm, 
    merged_df, 
    verbose=True, 
    agent=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True)

/Users/saikatbairagi/Git_repo/KN_GenAI/my-venv/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'agent': <AgentType.OPENAI_FUNCTIONS: 'openai-functions'>} which are no longer supported.
  warnings.warn(


In [34]:
response = agent.run("Which customer has placed the most orders?")
print(response)

The customer with the most orders is the customer with ID 789521, who placed 16 orders.


In [39]:
response = agent.run("Show me order count by items")
print(response)



> Entering new AgentExecutor chain...
Thought: To find the order count by items, I need to group the dataframe by `product_name` and count the number of orders for each product.
Action: python_repl_ast
Action Input: df.groupby('product_name')['order_id'].count()product_name
AM Biscuits 250     75
AM Biscuits 500    101
AM Biscuits 750     87
AM Butter 100       75
AM Butter 250      119
AM Butter 500      127
AM Curd 100         88
AM Curd 250         88
AM Curd 50          74
AM Ghee 100        121
AM Ghee 150        103
AM Ghee 250        108
AM Milk 100         87
AM Milk 250        104
AM Milk 500         92
AM Tea 100          96
AM Tea 250          91
AM Tea 500          86
Name: order_id, dtype: int64I now know the final answer.

Final Answer: The order count by items is as follows:

- AM Biscuits 250: 75 orders
- AM Biscuits 500: 101 orders
- AM Biscuits 750: 87 orders
- AM Butter 100: 75 orders
- AM Butter 250: 119 orders
- AM Butter 500: 127 orders
- AM Curd 100: 88 orders


In [40]:
response = agent.run("What is the top selling item and list down the orders")
print(response)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `To determine the top-selling item, we need to analyze the `order_qty` for each `product_name` and identify the product with the highest total `order_qty`. After identifying the top product, we'll list down the orders associated with that particular item.

Let's perform these steps:

1. Calculate the total `order_qty` for each `product_name`.
2. Identify the `product_name` with the highest total `order_qty`.
3. Filter the DataFrame to list orders associated with the top-selling product.

I'll execute these steps using the pandas DataFrame operations:

```python
# Step 1: Calculate total order_qty for each product
total_qty_per_product = df.groupby('product_name')['order_qty'].sum()

# Step 2: Identify the product with the highest total order_qty
top_selling_product = tota